In [0]:
#Long Short Term Memory

In [2]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as web
plt.style.use('fivethirtyeight')        

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [0]:
#pip install tensorflow

In [4]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential 
from keras.layers import Dense, LSTM

Using TensorFlow backend.


In [0]:
#https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92

In [6]:
df = pd.read_csv('https://raw.githubusercontent.com/smeet-kothari/SM-Portfolio-Manager/master/11-04-2018-TO-09-04-2020INFYEQN.csv')

df.head()

,Symbol,Series,Date,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,Average Price,Total Traded Quantity,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty
0,INFY,EQ,11-Apr-2018,1113.40,1118.00,1131.50,1116.50,1124.50,1124.25,1124.78,4512787,5.075892e+09,102586,3296304,73.04
1,INFY,EQ,12-Apr-2018,1124.25,1129.45,1172.75,1125.00,1164.05,1162.60,1157.71,8522183,9.866189e+09,216130,5624945,66.00
2,INFY,EQ,13-Apr-2018,1162.60,1174.00,1185.90,1150.25,1168.00,1171.45,1172.37,10613519,1.244292e+10,180965,5370870,50.60
3,INFY,EQ,16-Apr-2018,1171.45,1102.00,1140.00,1102.00,1134.50,1133.20,1125.66,12372542,1.392724e+10,201992,5012390,40.51
4,INFY,EQ,17-Apr-2018,1133.20,1127.00,1127.40,1115.00,1124.65,1125.20,1121.63,5198970,5.831318e+09,203792,3086531,59.37


In [7]:
df = df.iloc[ :, 8:9 ]

df.head()

,Close Price
0,1124.25
1,1162.60
2,1171.45
3,1133.20
4,1125.20


In [8]:
dataset = df.values

dataset[ :5 ]

array([[1124.25],
       [1162.6 ],
       [1171.45],
       [1133.2 ],
       [1125.2 ]])

In [0]:
#Scaling data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

In [10]:
# Training Dataset
training_length = math.ceil(0.8 *len(scaled_data))
train_data = scaled_data[ :training_length]
#split data into x_train and y_train
x_train = []
y_train = []

for i in range(60,len(train_data)):
    x_train.append(train_data[i-60:i,0])
    y_train.append(train_data[i,0])
    if i<= 60:
        print(x_train)
        print(y_train)
        print()

[array([0.65358334, 0.69551194, 0.70518778, 0.6633685 , 0.65462199,
       0.65544197, 0.66298584, 0.71284098, 0.72295414, 0.6866561 ,
       0.69305199, 0.71065435, 0.72022085, 0.73585525, 0.73317662,
       0.71732357, 0.70699175, 0.71781556, 0.69873722, 0.70436779,
       0.70141584, 0.7153556 , 0.72470344, 0.73044334, 0.72541409,
       0.71770623, 0.71803422, 0.71398896, 0.72967802, 0.71989286,
       0.7593615 , 0.7673427 , 0.75214563, 0.75466025, 0.74919368,
       0.7711693 , 0.75914284, 0.77947849, 0.75919751, 0.77734653,
       0.79374624, 0.80205543, 0.80462472, 0.80036079, 0.81818182,
       0.77980648, 0.8252337 , 0.81009129, 0.78188378, 0.78357842,
       0.78729569, 0.788717  , 0.81561253, 0.82151643, 0.81293391,
       0.83436287, 0.8536052 , 0.88367135, 0.90449899, 0.89515115])]
[0.8282403105012849]



In [11]:
#x_train and y_train to numpy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1) )
x_train.shape

(335, 60, 1)

In [0]:
#LSTM

model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1],x_train.shape[2]) ) )
model.add(LSTM(50, return_sequences=False) )
model.add(Dense(25))
model.add(Dense(1))

In [0]:
#Compile the Model

model.compile(optimizer='adam', loss='mean_squared_error')

In [14]:
#Train

model.fit(x_train,y_train,batch_size=1,epochs=1)

Epoch 1/1
335/335 [==============================] - 14s 41ms/step - loss: 0.0133


In [0]:
# Testing Data Set

test_data = scaled_data[training_length-60: , : ]

In [0]:
#Data Sets
x_test = []
y_test = dataset[training_length: , : ]

for i in range(60,len(test_data)):
    x_test.append(test_data[i-60:i, 0])

In [0]:
x_test = np.array(x_test)

In [0]:
#Reshape the dataset

x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1) )

In [0]:
#Get the model Predicted Value

predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [20]:
#Root Mean Sqaure error

rmse = np.sqrt( np.mean( predictions-y_test)**2 )
rmse

0.14492262236926856

In [21]:
train = df[ :training_length]
valid = df[training_length: ]

valid.head()

,Close Price
395,713.00
396,713.25
397,693.20
398,698.10
399,690.60


In [22]:
"""valid.loc['Predictions'] =predictions
#Visulization

plt.title('Model')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.plot(train.loc['Close Price'])
plt.show(valid.loc[['Close Price' , 'Predictions']])
plt.legend(['Train','Val','Predictions'], loc='upper_right' )"""

"valid.loc['Predictions'] =predictions\n#Visulization\n\nplt.title('Model')\nplt.xlabel('Date')\nplt.ylabel('Close Price')\nplt.plot(train.loc['Close Price'])\nplt.show(valid.loc[['Close Price' , 'Predictions']])\nplt.legend(['Train','Val','Predictions'], loc='upper_right' )"

In [0]:
new_df = pd.read_csv('https://raw.githubusercontent.com/smeet-kothari/SM-Portfolio-Manager/master/11-04-2018-TO-09-04-2020INFYEQN.csv')

In [24]:
new_df = new_df.iloc[ :, 8:9 ]

new_df.head()

,Close Price
0,1124.25
1,1162.60
2,1171.45
3,1133.20
4,1125.20


In [26]:
last_60_days = new_df[-60:].values

last_60_days_scaled = scaler.transform(last_60_days)

X_test = []
X_test.append(last_60_days_scaled)

X_test = np.array(x_test)

X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1) )

pred_price = model.predict(X_test)
pred_price = scaler.inverse_transform(pred_price)

print(pred_price[-1])

[606.9943]
